# **ETL - Telecom X**

### 📚 **Importación de librerías**

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 💿 **Extracción de datos**

In [68]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)
df.head(3)

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [69]:
df.shape

(7267, 6)

📝 **Notas**  
Los datos vienen en formato JSON, por lo que se debe realizar un ETL para obtener los datos en un formato que pueda ser utilizado para el análisis.

### 🔧 **Transformación de datos**

👨🏾‍⚖️ **Nomalización de columnas**  

In [70]:
df_customer = pd.json_normalize(df['customer'])
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])
df_phone = pd.json_normalize(df['phone'])

df_final = pd.concat([df['customerID'], df['Churn'], df_customer, df_phone, df_internet, df_account], axis=1)

df_final.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


### 🔍 **Análisis exploratorio de datos**

**Datos faltantes**

In [71]:
print('Datos faltantes: ')
df_final.isnull().sum()

Datos faltantes: 


customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64

**Tipo de datos**

In [72]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


**Conversión a numérico**  
La columna *Charges.Total* es de tipo string, por lo que se debe convertir a numérico. Identificare el problema de conversión y lo solucionare.

In [73]:
# Intentamos convertir a número
converted = pd.to_numeric(df_final['Charges.Total'], errors='coerce')

# Los valores que dan error se convierten a NaN
errores = df_final['Charges.Total'][converted.isna()]

print("Errores:", errores)

Errores: 975      
1775     
1955     
2075     
2232     
2308     
2930     
3134     
3203     
4169     
5599     
Name: Charges.Total, dtype: object


🕵️**Problema detectado**  
Los numeros que no pueden convertirse a enteros presentan espacios en blanco en la derecha. Procederé a eliminar los espacios en blanco.

In [74]:
df_final['Charges.Monthly'] = pd.to_numeric(df_final['Charges.Monthly'].astype(str).str.strip(), errors='coerce')
df_final['Charges.Total'] = pd.to_numeric(df_final['Charges.Total'].astype(str).str.strip(), errors='coerce')

In [75]:
print('Tipo de datos de la columna Charges.Monthly: ', df_final['Charges.Monthly'].dtype)

print('Cantidad de nulos: ', df_final['Charges.Monthly'].isnull().sum())

Tipo de datos de la columna Charges.Monthly:  float64
Cantidad de nulos:  0


**Corrección de los nombres de las columnas**

In [76]:
# Elimino columna Customer id 
df_final.drop(columns=['customerID'], inplace=True)

In [77]:
rename_dict = {
    'Churn': 'Churn',
    'gender': 'Género',
    'SeniorCitizen': 'Adulto Mayor',
    'Partner': 'Pareja',
    'Dependents': 'Dependientes',
    'tenure': 'Tiempo de Servicio (meses)',
    'PhoneService': 'Servicio Telefónico',
    'MultipleLines': 'Múltiples Líneas',
    'InternetService': 'Servicio de Internet',
    'OnlineSecurity': 'Seguridad Online',
    'OnlineBackup': 'Respaldo Online',
    'DeviceProtection': 'Protección de Dispositivo',
    'TechSupport': 'Soporte Técnico',
    'StreamingTV': 'Streaming de TV',
    'StreamingMovies': 'Streaming de Películas',
    'Contract': 'Tipo de Contrato',
    'PaperlessBilling': 'Factura Electrónica',
    'PaymentMethod': 'Método de Pago',
    'Charges.Monthly': 'Cobro Mensual',
    'Charges.Total': 'Cobro Total'
}

df_final.rename(columns=rename_dict, inplace=True)


### 📌 **Nueva columna - Cuentas diarias**

Utilizo la facturación mensual para calcular el valor diario, proporcionando una visión más detallada del comportamiento de los clientes a lo largo del tiempo.

In [78]:
df_final['Cuentas Diarias'] = np.round(df_final['Cobro Mensual'] / 30, 2)

df_final.head(3)

,Churn,Género,Adulto Mayor,Pareja,Dependientes,Tiempo de Servicio (meses),Servicio Telefónico,Múltiples Líneas,Servicio de Internet,Seguridad Online,...,Protección de Dispositivo,Soporte Técnico,Streaming de TV,Streaming de Películas,Tipo de Contrato,Factura Electrónica,Método de Pago,Cobro Mensual,Cobro Total,Cuentas Diarias
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


### 📁 **Guardar datos**

In [79]:
df_final.to_csv("../data/df_limpio.csv", index=False)

### 📏 **Normalización de features**

In [80]:
# Cargamos el dataset limpio
df_limpio = pd.read_csv("../data/df_limpio.csv")

df_limpio.head(3)

,Churn,Género,Adulto Mayor,Pareja,Dependientes,Tiempo de Servicio (meses),Servicio Telefónico,Múltiples Líneas,Servicio de Internet,Seguridad Online,...,Protección de Dispositivo,Soporte Técnico,Streaming de TV,Streaming de Películas,Tipo de Contrato,Factura Electrónica,Método de Pago,Cobro Mensual,Cobro Total,Cuentas Diarias
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


In [81]:
# Tipos de datos
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Churn                       7043 non-null   object 
 1   Género                      7267 non-null   object 
 2   Adulto Mayor                7267 non-null   int64  
 3   Pareja                      7267 non-null   object 
 4   Dependientes                7267 non-null   object 
 5   Tiempo de Servicio (meses)  7267 non-null   int64  
 6   Servicio Telefónico         7267 non-null   object 
 7   Múltiples Líneas            7267 non-null   object 
 8   Servicio de Internet        7267 non-null   object 
 9   Seguridad Online            7267 non-null   object 
 10  Respaldo Online             7267 non-null   object 
 11  Protección de Dispositivo   7267 non-null   object 
 12  Soporte Técnico             7267 non-null   object 
 13  Streaming de TV             7267 

In [82]:
df_limpio.Churn.isna().sum()

np.int64(224)

In [83]:
# Variables que necesitan ser convertidas a 0 y 1
var_binaria = ['Género', 'Pareja', 'Dependientes', 'Servicio Telefónico', 'Factura Electrónica', 'Churn']

# Variables que necesitan ser convertidas a dummies
var_dummies = ['Múltiples Líneas', 'Servicio de Internet',  'Seguridad Online', 'Respaldo Online', 'Protección de Dispositivo', 'Soporte Técnico', 'Streaming de TV', 'Streaming de Películas', 'Tipo de Contrato', 'Método de Pago']

In [84]:
# 0 y 1 con dummies y drop=first
for col in var_binaria:
    df_limpio[col] = pd.get_dummies(df_limpio[col], drop_first=True)

In [85]:
# Dummies
df_limpio = pd.get_dummies(df_limpio, columns=var_dummies)

In [90]:
# Guardar el datafram transformado
df_limpio.to_csv("../data/df_estandarizado.csv", index=False)